## Environments

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
from pycaret.classification import *
import warnings
warnings.filterwarnings(action='ignore')

PATH = 'C:/Users/JYW/Desktop/Github/repository/Dacon/UserProblem_from_ServiceQuality_LG/data/'

## Train datasets

In [5]:
train_err = pd.read_csv(PATH+'train_err_data.csv')

print(f"train_err shape : {train_err.shape}")
train_err.head()

train_err shape : (16554663, 6)


,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [6]:
train_quality  = pd.read_csv(PATH+'train_quality_data.csv')

print(f"train_quality shape : {train_quality.shape}")
train_quality.head()

train_quality shape : (828624, 16)


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [7]:
train_problem = pd.read_csv(PATH+'train_problem_data.csv')

print(f"train_problem shape : {train_problem.shape}")
train_problem.head()

train_problem shape : (5429, 2)


,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000


## Data transform for Baseline

In [8]:
id_error = train_err[['user_id','errtype']].values
error = np.zeros((15000,42))
err = np.zeros((15000,42))
for person_idx, err in tqdm(id_error):
    error[person_idx - 10000,err - 1] += 1

problem = np.zeros(15000)
problem[train_problem.user_id.unique()-10000] = 1

train = pd.DataFrame(data=error)
train['problem'] = problem
del error, problem

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:44<00:00, 375520.38it/s]


## Baseline models

In [9]:
clf = setup(data = train, target = 'problem')

best_5 = compare_models(sort = 'AUC', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7812,0.8069,0.4916,0.7717,0.6005,0.4599,0.4821,8.8770
gbc,Gradient Boosting Classifier,0.7770,0.8014,0.4577,0.7868,0.5784,0.4410,0.4711,0.9120
lightgbm,Light Gradient Boosting Machine,0.7796,0.8003,0.5004,0.7590,0.6029,0.4592,0.4784,0.3410
rf,Random Forest Classifier,0.7763,0.7977,0.4702,0.7718,0.5843,0.4434,0.4689,0.7010
et,Extra Trees Classifier,0.7761,0.7958,0.4811,0.7622,0.5897,0.4464,0.4688,0.7620
ada,Ada Boost Classifier,0.7701,0.7894,0.4563,0.7608,0.5703,0.4265,0.4526,0.2490
xgboost,Extreme Gradient Boosting,0.7741,0.7872,0.5272,0.7228,0.6095,0.4560,0.4674,0.9610
lda,Linear Discriminant Analysis,0.7437,0.7341,0.3145,0.7966,0.4508,0.3225,0.3819,0.0530
lr,Logistic Regression,0.7536,0.7339,0.3823,0.7632,0.5091,0.3682,0.4075,1.6090
qda,Quadratic Discriminant Analysis,0.7442,0.7309,0.3328,0.7775,0.4635,0.3301,0.3821,0.0240
